<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#make-passphrase-hash" data-toc-modified-id="make-passphrase-hash-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>make passphrase hash</a></span></li><li><span><a href="#digest-vs.-hexdigest" data-toc-modified-id="digest-vs.-hexdigest-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>digest vs. hexdigest</a></span><ul class="toc-item"><li><span><a href="#Manual-conversion-for-understanding" data-toc-modified-id="Manual-conversion-for-understanding-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Manual conversion for understanding</a></span><ul class="toc-item"><li><span><a href="#Step-1---convert-hex-characters-to-binary-numbers" data-toc-modified-id="Step-1---convert-hex-characters-to-binary-numbers-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Step 1 - convert hex characters to binary numbers</a></span></li><li><span><a href="#Step-2---convert-binary-string-list-to-byte-array" data-toc-modified-id="Step-2---convert-binary-string-list-to-byte-array-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Step 2 - convert binary string list to byte array</a></span></li></ul></li><li><span><a href="#built-in-Python-methods" data-toc-modified-id="built-in-Python-methods-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>built-in Python methods</a></span><ul class="toc-item"><li><span><a href="#bytearray.fromhex" data-toc-modified-id="bytearray.fromhex-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>bytearray.fromhex</a></span></li><li><span><a href="#binascii.unhexlify" data-toc-modified-id="binascii.unhexlify-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>binascii.unhexlify</a></span></li></ul></li></ul></li><li><span><a href="#hash-values" data-toc-modified-id="hash-values-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>hash values</a></span></li></ul></div>

# imports

In [ ]:
# use Python 3.6 secrets package
import binascii
import hashlib
import math
import secrets
import sys

# make passphrase hash

In [ ]:
# what is our secret?
passphrase = "fakedata"

# get hasher
sha256_instance = hashlib.sha256()

# encode to utf-8, then put the secret in the SHA256 hasher.
encoded_passphrase = passphrase.encode( "utf-8" )
sha256_instance.update( encoded_passphrase )

# get hash as digest (byte array)
passphrase_hash = sha256_instance.digest()
print( "digest(): {}".format( passphrase_hash ) )

# get hash as hexdigest (byte array)
passphrase_hash_hex = sha256_instance.hexdigest()
print( "hexdigest(): {}".format( passphrase_hash_hex ) )

# digest vs. hexdigest

- Back to [Table of Contents](#Table-of-Contents)

The byte array we used to hash (output by the `digest()` method) can be represented as the hexadecimal number `hexdigest()` outputs, but that is just a more readable encoding of the underlying binary hash output - to convert to hex, the program takes each 4-bit chunk, from left to right, and converts the 4 binary/base-2 bits to a hexadecimal number, then it stores that hex digit's character in a string.

If you just convert that back to binary, you don’t get the same bit stream - you get the ascii/utf-8 character codes, one to a byte.  To get the same output, you need to convert back to binary the reverse of the way the hex values were generated.

So, to create a byte array that will result in the same hash as our code:

- take each hex digit and convert it to a 4-digit base 2/binary number (left padded with zeros for small values).
- concatenate all the digits together into a big long string of bits, then, from left to right, store each set of 8 bits as a byte in the byte array/binary data blob that you’ll use as the HMAC secret.

## Manual conversion for understanding

- Back to [Table of Contents](#Table-of-Contents)

### Step 1 - convert hex characters to binary numbers

- Back to [Table of Contents](#Table-of-Contents)

1) Take each hex digit and convert it to a 4-digit base 2/binary number (left padded with zeros for small values):

In [ ]:
# declare variables
four_bit_binary_list = []

# loop over each.
for item in passphrase_hash_hex:
    
    item_int = int( item, 16 )
    item_bin = "{0:b}".format( item_int )
    item_bin_string = str( item_bin )
    item_bin_string_padded = item_bin_string.zfill( 4 )
    item_bin_length = len( item_bin )
    
    print( "{} = {} ( {} - {} - len: {} )".format( item, item_int, item_bin, item_bin_string_padded, item_bin_length ) )
    
    # add to list.
    four_bit_binary_list.append( item_bin_string_padded )
    
#-- END loop over passphrase_hash --#

### Step 2 - convert binary string list to byte array

- Back to [Table of Contents](#Table-of-Contents)

- concatenate all the digits together into a big long binary stream.
- then, from left to right, store each set of 8 bits as a byte in the byte array/binary data blob that you’ll use as the HMAC secret.

In [ ]:
# join the list into one big string.
binary_string = "".join( four_bit_binary_list )

# Python 3 way:
binary_bytes = int( binary_string, 2 ).to_bytes( ( len( binary_string ) + 7 ) // 8, byteorder = 'big' )
print( "int().to_bytes()..: {}".format( binary_bytes ) )

# Python 2 compatible:
def bitstring_to_bytes(s):
    v = int(s, 2)
    b = bytearray()
    while v:
        b.append(v & 0xff)
        v >>= 8
    return bytes(b[::-1])

#-- END function bitstring_to_bytes() --#

py2_binary_bytes = bitstring_to_bytes( binary_string )
print( "bitstring_to_bytes: {}".format( py2_binary_bytes ) )

print( "digest() output...: {}".format( passphrase_hash ) )

## built-in Python methods

- Back to [Table of Contents](#Table-of-Contents)

Python provides built-in functions to accomplish this.

### bytearray.fromhex

- Back to [Table of Contents](#Table-of-Contents)

From: https://docs.python.org/3.6/library/stdtypes.html#bytearray-objects

Since 2 hexadecimal digits correspond precisely to a single byte, hexadecimal numbers are a commonly used format for describing binary data. Accordingly, the bytearray type has an additional class method to read data in that format:

classmethod fromhex(string)

    This bytearray class method returns bytearray object, decoding the given string object. The string must contain two hexadecimal digits per byte, with ASCII whitespace being ignored.

    >>> bytearray.fromhex('2Ef0 F1f2  ')
    bytearray(b'.\xf0\xf1\xf2')

A reverse conversion function exists to transform a bytearray object into its hexadecimal representation.

hex()

    Return a string object containing two hexadecimal digits for each byte in the instance.

    >>> bytearray(b'\xf0\xf1\xf2').hex()
    'f0f1f2'

More information:

- https://docs.python.org/3.6/c-api/bytearray.html
- https://docs.python.org/3.6/library/stdtypes.html#bytearray-objects


In [ ]:
test_bytes = bytearray.fromhex( passphrase_hash_hex )
print( "passphrase_hash_hex = {}".format( passphrase_hash_hex ) )
print( "bytearray.fromhex( passphrase_hash_hex ) = {}".format( test_bytes ) )

### binascii.unhexlify

- Back to [Table of Contents](#Table-of-Contents)

From: https://docs.python.org/3/library/binascii.html#binascii.b2a_hex 

binascii.b2a_hex(data)
binascii.hexlify(data)

    Return the hexadecimal representation of the binary data. Every byte of data is converted into the corresponding 2-digit hex representation. The returned bytes object is therefore twice as long as the length of data.

binascii.a2b_hex(hexstr)
binascii.unhexlify(hexstr)

    Return the binary data represented by the hexadecimal string hexstr. This function is the inverse of b2a_hex(). hexstr must contain an even number of hexadecimal digits (which can be upper or lower case), otherwise an Error exception is raised.


In [ ]:
binascii_test_bytes = binascii.unhexlify( passphrase_hash_hex )
print( "passphrase_hash_hex = {}".format( passphrase_hash_hex ) )
print( "bytearray.fromhex( passphrase_hash_hex ) = {}".format( binascii_test_bytes ) )

# hash values

In [ ]:
# digest() output
import hmac

# byte array secret and encoded message (each is required)
message = "123456789"
encoded_message = message.encode( "utf-8" )
hmac_key = passphrase_hash
hmac_instance = hmac.new( hmac_key, encoded_message, digestmod = hashlib.sha256 )
hashed_value = hmac_instance.hexdigest()
print( "Hash of {}: {}".format( message, hashed_value )  )

In [ ]:
# reconstructed binary_bytes
import hmac

# byte array secret and encoded message (each is required)
message = "123456789"
encoded_message = message.encode( "utf-8" )
hmac_key = binary_bytes
hmac_instance = hmac.new( hmac_key, encoded_message, digestmod = hashlib.sha256 )
hashed_value = hmac_instance.hexdigest()
print( "Hash of {}: {}".format( message, hashed_value )  )

In [ ]:
# reconstructed py2_binary_bytes
import hmac

# byte array secret and encoded message (each is required)
message = "123456789"
encoded_message = message.encode( "utf-8" )
hmac_key = py2_binary_bytes
hmac_instance = hmac.new( hmac_key, encoded_message, digestmod = hashlib.sha256 )
hashed_value = hmac_instance.hexdigest()
print( "Hash of {}: {}".format( message, hashed_value )  )